In [3]:
using CSV, DataFrames, JuMP, Gurobi, LinearAlgebra, Random

In [4]:
# Constants

nP = 10 # number of people
nT = 210 # number of timeslots in a week
# Organized in 30 minute interals from 9am-12pm, Sunday-Saturday
# 30 intervals/day * 7 days/week
cM = 1e7 # multiplier for the minimal matching objective

print("")

In [5]:
# Load Data

# 0-1 matrix of whether person can mentor/mentee at time T
A = Matrix(CSV.read("schedules.csv", DataFrame, header=0))  # nP x nT


# for these, what if someone is only a mentor or only a mentee? would it be zero?
aR = Matrix(CSV.read("max_mentoring.csv")) # nP x 1: max num of session person can mentor
aE = Matrix(CSV.read("max_menteeing.csv")) # nP x 1: max num of session person can mentee (can we remove this?)
r  = Matrix(CSV.read("rank.csv")) # nP x 1, rank

# Don't we just need one per person
pR = Matrix(CSV.read("mentor_poomsae.csv")) # nP x 1: 0-1 does mentor prefer poomsae 
pE = Matrix(CSV.read("mentee_poomsae.csv")) # nP x 1: 0-1 does mentee prefer poomsae
sR = Matrix(CSV.read("mentor_sparring.csv")) # nP x 1: 0-1 does mentor prefer sparring
sE = Matrix(CSV.read("mentee_sparring.csv")) # nP x 1: 0-1 does mentee prefer sparring

print("")

LoadError: ArgumentError: "schedules.csv" is not a valid file or doesn't exist

In [4]:
# Create Cost Matrix

alpha = 1
beta  = 1

C = zeros(nP, nP)

for i = 1:nP, j = 1:nP
    C[i, j] += alpha * (r[i]-r[j]-2+(r[j] >= 9))
    C[i, j] -= beta  * (pR[i]*pE[j]+sR[i]*sE[j]-pR[i]*sR[i]-pE[j]*sE[j])
    C[i, j] -= 1
end

In [5]:
# Rank Constraint Matrix

R = zeros(nP, nP, nT)

for i = 1:nP, j = 1:nP, T = 1:nT
    if r[i] >= r[j]+2 || (r[i] >= r[j]+1 && r[j] >= 9)
        R[i, j, T] = 1
    end
end

In [6]:
m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

print("")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-23


In [7]:
@variable(m, 0 <= x[1:nP, 1:nP, 1:nT] <= 1)
@variable(m, 0 <= t[1:nP] <= 1, Int)

print("")

In [8]:
# Rank Constraint
@constraint(m, x .<= R)

# Maximum Constraints
for P = 1:nP
    @constraint(m, sum(x[i, P, T] for i=1:nP, T = 1:nT) <= aR[P])
    @constraint(m, sum(x[P, i, T] for i=1:nP, T = 1:nT) <= aE[P])
end

# Availability Constraints
for i = 1:nP, j = i+1:nP, T=1:nT
    @constraint(m, x[i,j,T] <= A[i,T]*A[j,T])
end

# Single Mentor Constraint
for i = 1:nP, j = i+1:nP, k = 1:nP, T=1:nT
    @constraint(m, x[i,k,T]+x[j,k,T] <= 1)
end

# Making t work
for P = 1:nP
    @constraint(m, sum(x[i,P,T] for i=1:nP, T=1:nT) >= t[P])
    @constraint(m, 0.01*sum(x[i,P,T] for i=1:nP, T=1:nT) <= t[P])
end

print("")

In [9]:
@objective(m, Min, cM*sum((1-t[P]) for P=1:nP) + sum(x[i,j,T]*C[i,j] for i=1:nP, j=1:nP, T=1:nT))
print("")